In [1]:
%pip install --upgrade python-dotenv nest_asyncio pydantic google-genai requests pandas

from IPython.display import clear_output ; clear_output()

In [3]:
import os
import random
import asyncio

from dotenv import load_dotenv
import nest_asyncio

from textwrap import dedent
from IPython.display import display, Markdown
def md(str): display(Markdown(str))

from pydantic import BaseModel, Field
from enum import Enum

import pandas as pd

from google import genai
from google.genai import types

load_dotenv()

nest_asyncio.apply()

_gemini_client = genai.Client(api_key=os.getenv('GEMINI_API_KEY'))

G25PRO = 'gemini-2.5-pro-preview-03-25'
G25FLASH = 'gemini-2.5-flash-preview-04-17'

async def gemini(
        prompt: str,
        pro: bool = False,
        max_tokens: int = None, temperature: float = None,
        budget: int = None,
        schema: BaseModel = None,):
    config = {}
    if max_tokens is not None:
        config['max_output_tokens'] = max_tokens
    if temperature is not None:
        config['temperature'] = temperature
    if budget is not None:
        config['thinking_config'] = {'thinking_budget': budget}
    if schema is not None:
        config['response_mime_type'] = 'application/json'
        config['response_schema'] = schema
    
    response = await _gemini_client .aio.models.generate_content(
        model=(G25PRO if pro else G25FLASH),
        contents=prompt,
        config=config,
    )
    
    if schema is not None:
        return response.parsed
    else:
        return response.text

In [6]:
class Location(str, Enum):
    NATURE = 'nature'
    OFFICE = 'office'
    PUB = 'pub'
    STREET = 'street'
    HOME = 'home'

class Mood(str, Enum):
    HAPPY = 'happy'
    SAD = 'sad'
    CALM = 'calm'
    SCARY = 'scary'

class TimeOfDay(str, Enum):
    MORNING = 'morning'
    AFTERNOON = 'afternoon'
    EVENING = 'evening'
    NIGHT = 'night'

class Gender(str, Enum):
    MALE = 'male'
    FEMALE = 'female'
    OTHER = 'other'

class Person(BaseModel):
    name: str = Field(description="Name of the person")
    age: int = Field(description="Age of the person")
    gender: Gender = Field(description="Gender of the person")

class Story(BaseModel):
    people: list[Person] = Field(description="List of people in the story")
    location: Location = Field(description="Location of the story")
    time_of_day: TimeOfDay = Field(description="Time of day in the story")
    mood: Mood = Field(description="Mood of the story")

async def create_story():
    num_people = random.randint(2, 4)
    location = random.choice([loc.value for loc in Location])
    mood = random.choice([mood.value for mood in Mood])
    time_of_day = random.choice([time.value for time in TimeOfDay])

    prompt = dedent(f"""
        - Write a very short story depicting a situation.
        - Describe, the people (with their name, age, gender ({', '.join([gender.value for gender in Gender])}).
        - Describe the location, the time of day, and the mood with sufficient detail,
          to allow the reader to discern them clearly and unambiguously. The reader will be tested
          on their ability to identify the people, location, time of day, and mood from the story,
          so be sure to include enough detail for them to do so, and help them out by ensuring that
          the story describes them clearly enough to make them easy to identify.
        - Don't use any special characters or markdown formatting.
        - The story should be roughly 350 tokens long.

        <story-specification>
          - Number of people: {num_people}
          - Location: {location}
          - Time of day: {time_of_day}
          - Mood: {mood}
        </story-specification>
    """)

    response = await gemini(prompt, max_tokens=2500, temperature=0.8, budget=2000)

    return prompt, response

async def parse_story(story):
    prompt = dedent(f"""
        Parse the story and extract the following information:
        - People: name, age, gender ({', '.join([gender.value for gender in Gender])})
        - Location: {', '.join([loc.value for loc in Location])}
        - Time of day: {', '.join([time.value for time in TimeOfDay])}
        - Mood: {', '.join([mood.value for mood in Mood])}

    <story>
      {story}
    </story>
    """)
    response = await gemini(prompt, schema=Story, budget=3000, temperature=0.3)

    people = sorted(response.people, key=lambda x: x.name)
    return dedent(f"""\
    PEOPLE:
    {'\n    '.join([f"  - {person.name}, {person.age}, {person.gender.value}" for person in people])}
    LOCATION: {response.location.value}
    TIME_OF_DAY: {response.time_of_day.value}
    MOOD: {response.mood.value}
    """)

In [7]:

prompt, response = await create_story()
parsed = await parse_story(response)

md(f"### Prompt\n\n```{prompt}```\n\n### Response\n\n{response}\n\n### Parsed\n\n```\n{parsed}\n```")

### Prompt

```
- Write a very short story depicting a situation.
- Describe, the people (with their name, age, gender (male, female, other).
- Describe the location, the time of day, and the mood with sufficient detail,
  to allow the reader to discern them clearly and unambiguously. The reader will be tested
  on their ability to identify the people, location, time of day, and mood from the story,
  so be sure to include enough detail for them to do so, and help them out by ensuring that
  the story describes them clearly enough to make them easy to identify.
- Don't use any special characters or markdown formatting.
- The story should be roughly 350 tokens long.

<story-specification>
  - Number of people: 4
  - Location: street
  - Time of day: morning
  - Mood: sad
</story-specification>
```

### Response

The grey light of early morning barely illuminated Elm Street. A heavy stillness hung in the air, matching the weight in their chests. It wasn't just the cool, damp air that made them shiver on the sidewalk. Elara, a woman of 45, stood clutching her worn coat collar tightly. Her eyes, usually bright, were dull and fixed on the cracked pavement near an overflowing bin. Beside her, Thomas, a man of 50, placed a comforting but weak hand on her back. He looked older than his years this morning, his face etched with sorrow lines that seemed deeper than usual. Their son, Liam, a quiet male of 17, stood slightly apart, kicking aimlessly at a loose stone near the curb, his young face a mask of grief that was hard to see. Little Chloe, a girl of 10, huddled close to her father's leg, her small shoulders shaking silently, tears tracing clean paths on her cheeks. The usual morning sounds of the street were absent for them, replaced by the quiet hum of their shared sadness. They stood on the sidewalk, a small, mournful cluster against the indifferent start of the day, waiting for something they dreaded near the bus stop sign. The emptiness felt palpable, a physical presence on the street corner, heavy and cold.

### Parsed

```
PEOPLE:
  - Chloe, 10, female
  - Elara, 45, female
  - Liam, 17, male
  - Thomas, 50, male
LOCATION: street
TIME_OF_DAY: morning
MOOD: sad

```

In [8]:
async def a_gen_item():
    _, response = await create_story()
    parsed = await parse_story(response)
    return response, parsed

stories = pd.DataFrame(
    await asyncio.gather(*[a_gen_item() for _ in range(100)]),
    columns=['input', 'output'],
)

display(stories.head(3).style.set_properties(
    **{'text-align': 'left', 'vertical-align': 'top', 'white-space': 'pre-wrap', 'width': '50%'},
))

,input,output
0,"Anya, a 35 year old female, held the small hand of Clara, her 8 year old female daughter. Beside them walked Ben, a 40 year old male, Anya's husband. They strolled slowly down the quiet street. The only light came from the old-fashioned lampposts that cast pools of warm, yellow light onto the smooth pavement. Above, the sky was a deep navy blue, dotted with distant stars. The air was cool and still, carrying only the faint sound of their footsteps and the occasional rustle of leaves in the gentle breeze. No cars passed, no loud noises disturbed the peace. Clara hummed a soft, tuneless melody. Ben had his hands in his pockets, looking up at the sky occasionally. Anya smiled faintly, feeling the quiet contentment of the moment. It was late, most shops were dark, their windows reflecting the streetlights like sleepy eyes. The familiar buildings stood silent watch. They weren't rushing anywhere, just enjoying the quiet end to the day. The peaceful atmosphere wrapped around them like a soft blanket.","PEOPLE: - Anya, 35, female - Ben, 40, male - Clara, 8, female LOCATION: street TIME_OF_DAY: night MOOD: calm"
1,"Sarah, 35, female, leaned back on the soft grass, a wide smile of pure contentment on her face. Beside her, David, 38, male, gently tossed a small stone he'd found. He chuckled softly as Lily, 7, female, excitedly recounted a story about a butterfly she'd seen earlier. Lily, her bright eyes reflecting the changing light, sat cross-legged between them, occasionally pointing at interesting cloud shapes. They were in a sprawling, open meadow, surrounded by the quiet dignity of ancient trees at its edge. The air was sweet with the perfume of clover and pine needles carried on a light breeze, and it held the subtle chill that comes with the setting sun. Distant sounds of the world felt far away, replaced by the gentle hum of insects and the distant hoot of an early owl. The sun was a magnificent orb descending towards the horizon, bathing the entire scene in a warm, golden light that deepened into fiery oranges and soft purples. Long, lazy shadows stretched across the uneven ground, reaching like fingers towards the forest. A feeling of deep peace and unadulterated happiness permeated the atmosphere. David reached over and took Sarah's hand, squeezing it gently. Lily began to hum a little tune, entirely at ease. This simple moment, shared under the vast, colourful sky, felt perfect to all three of them. Their relaxed postures and easy laughter spoke volumes about the joy of being together in this tranquil place as the day drew to a close.","PEOPLE: - David, 38, male - Lily, 7, female - Sarah, 35, female LOCATION: nature TIME_OF_DAY: evening MOOD: happy"
2,"Evelyn, a woman aged 65, sat quietly in a worn armchair by the large window. Across from her was Thomas, a man of 70, holding a simple mug. The room was dimly lit by sunlight filtering through the high, slightly dusty windows. It was early, just after nine o'clock, and the air held a stillness uncommon later in the day. The place was The Anchor, a pub known for its lively evenings, but now it was almost deserted. The faint scent of old wood and yesterday's spilled drinks was present, but it was overlaid by the smell of brewing coffee from behind the empty bar. Thomas took a slow sip from his mug, his movements unhurried. Evelyn watched the street outside, a soft, peaceful expression on her face. There was no rush, no loud voices, just the quiet hum of the refrigerator and the distant sounds of the waking town. A deep sense of calm settled over their corner of the pub.","PEOPLE: - Evelyn, 65, female - Thomas, 70, male LOCATION: pub TIME_OF_DAY: morning MOOD: calm"
